In [2]:
import random
import ipywidgets as widgets
from IPython.display import display

class Player:
    def __init__(self, name, money=1000000, allowances=8, upgrades=[]):
        self.name = name
        self.money = money
        self.allowances = allowances
        self.upgrades = upgrades

def roll_dice():
    return random.randint(1, 6) + random.randint(1, 6)

def buy_allowances(player1, player2, price, num_allowances):
    if player1.money >= price:
        player1.money -= price
        player1.allowances += num_allowances
        player2.money += price
        player2.allowances -= num_allowances

def buy_upgrade(player, upgrade):
    if upgrade not in player.upgrades:
        if player.money >= upgrade_costs[upgrade]:
            player.money -= upgrade_costs[upgrade]
            player.upgrades.append(upgrade)
            return f"{player.name} purchased {upgrade}."
        else:
            return f"{player.name} doesn't have enough money to buy {upgrade}."
    else:
        return f"{player.name} already has {upgrade}."

def trade(player1, player2, price, num_allowances):
    response1 = buy_allowances(player1, player2, price, num_allowances)
    response2 = buy_allowances(player2, player1, price, num_allowances)
    return f"{player1.name}: {response1}, {player2.name}: {response2}"

def reduce_emissions(player):
    emissions = roll_dice()
    if "Mega upgrade" in player.upgrades:
        roll = roll_dice()
        emissions = min(roll, emissions)
    if "Transition to clean energy" in player.upgrades:
        return roll_dice()
    return emissions

# Create player objects
players = [Player(f"Player {i + 1}") for i in range(4)]

# Create widgets
player_dropdown = widgets.Dropdown(options=[player.name for player in players], description="Player: ")
buy_sell_button = widgets.Button(description="Buy/Sell Allowances")
buy_sell_output = widgets.Output()
upgrade_button = widgets.Button(description="Buy Upgrade")
upgrade_output = widgets.Output()

# Game variables
rounds = 5
round_num = 2
upgrade_costs = {
    "Invest in clean energy": 200000,
    "Upgrade your plant's technology": 300000,
    "Mega upgrade": 400000,
    "Transition to clean energy": 600000
}

def play_round():
    global round_num

    player_name = player_dropdown.value
    current_player = None

    for player in players:
        if player.name == player_name:
            current_player = player
            break

    emissions = reduce_emissions(current_player)
    with buy_sell_output:
        buy_sell_output.clear_output()
        display(widgets.HTML(f"<strong>{current_player.name}'s Turn</strong>"))
        display(widgets.HTML(f"{current_player.name} rolled {emissions} units of pollution."))
        if emissions > current_player.allowances:
            fines = (emissions - current_player.allowances) * 100000
            current_player.money -= fines
            current_player.allowances = 0
            display(widgets.HTML(f"{current_player.name} was fined ${fines}."))
        else:
            current_player.allowances -= emissions
        display(widgets.HTML(f"{current_player.name} has ${current_player.money} and {current_player.allowances} allowances left."))

def buy_sell_allowances(_):
    global round_num
    if round_num <= rounds:
        with buy_sell_output:
            buy_sell_output.clear_output()
            trade_players = [player.name for player in players if player.name != player_dropdown.value]
            trade_dropdown = widgets.Dropdown(options=trade_players, description="Trade with: ")
            price_input = widgets.BoundedIntText(value=0, min=0, description="Price: $")
            num_allowances_input = widgets.BoundedIntText(value=0, min=0, description="Number of Allowances: ")
            trade_button = widgets.Button(description="Trade")
            trade_response_output = widgets.Output()

            def trade(_):
                trade_player_name = trade_dropdown.value
                price = price_input.value
                num_allowances = num_allowances_input.value

                current_player_name = player_dropdown.value
                current_player = None
                trade_player = None

                for player in players:
                    if player.name == current_player_name:
                        current_player = player
                    if player.name == trade_player_name:
                        trade_player = player

                if current_player and trade_player:
                    response = trade(current_player, trade_player, price, num_allowances)
                    with trade_response_output:
                        trade_response_output.clear_output()
                        display(widgets.HTML(response))
                else:
                    with trade_response_output:
                        trade_response_output.clear_output()
                        display(widgets.HTML("Invalid players selected for trade."))

            trade_button.on_click(trade)

            display(trade_dropdown, price_input, num_allowances_input, trade_button, trade_response_output)

def buy_upgrade_button(_):
    with upgrade_output:
        upgrade_output.clear_output()
        current_player_name = player_dropdown.value
        current_player = None

        for player in players:
            if player.name == current_player_name:
                current_player = player
                break

        if current_player:
            upgrade_options = [k for k, v in upgrade_costs.items() if v <= current_player.money and k not in current_player.upgrades]
            upgrade_dropdown = widgets.Dropdown(options=upgrade_options, description="Choose Upgrade: ")

            upgrade_button = widgets.Button(description="Buy Upgrade")
            upgrade_response_output = widgets.Output()

            def buy_upgrade(_):
                if upgrade_dropdown.value:
                    response = buy_upgrade(current_player, upgrade_dropdown.value)
                    with upgrade_response_output:
                        upgrade_response_output.clear_output()
                        display(widgets.HTML(response))

            upgrade_button.on_click(buy_upgrade)
            display(upgrade_dropdown, upgrade_button, upgrade_response_output)

# Widget interactions
buy_sell_button.on_click(buy_sell_allowances)
upgrade_button.on_click(buy_upgrade_button)

# Display widgets
with buy_sell_output:
    display(widgets.HTML(f"<strong>Round {round_num}</strong>"))
with upgrade_output:
    display(widgets.HTML(f"<strong>Round {round_num}</strong>"))

widgets.VBox([player_dropdown, buy_sell_button, buy_sell_output, upgrade_button, upgrade_output])


In [3]:
import numpy as np
arr=np.arange(4,8,3)
print(arr)

[4 7]
